<a href="https://colab.research.google.com/github/dude123studios/AdvancedDeepLearning/blob/main/Transfer_Learning_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras import  layers, models

strategy = tf.distribute.MirroredStrategy(devices=['GPU:0'])
with strategy.scope():
    base_model = InceptionV3(weights='imagenet',include_top=False)

    x = base_model.output
    x = layers.Dense(1024,activation='relu')(x)
    preds = layers.Dense(200,activation='softmax')(x)

    model = models.Model(inputs=base_model.input,outputs=preds)
    base_model.trainable = False
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy')

#use actual data and stuff
model.fit()

with strategy.scope():
    for layers in model.layers[:172]:
        layers.trainable = False
    for layers in model,layers[172:]:
        layers.trainable = True
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.0001,momentum=0.9),
                  loss='categorical_crossentropy',metrics=['accuracy'])
#use rest of dataset 
model.fit()